# Publications markdown generator for academicpages

Takes a set of bibtex of publications and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). 

The core python code is also in `pubsFromBibs.py`. 
Run either from the `markdown_generator` folder after replacing updating the publist dictionary with:
* bib file names
* specific venue keys based on your bib file preferences
* any specific pre-text for specific files
* Collection Name (future feature)

TODO: Make this work with other databases of citations, 
TODO: Merge this with the existing TSV parsing solution

In [6]:
from pybtex.database.input import bibtex
import pybtex.database.input.bibtex 
from time import strptime
import string
import html
import os
import re

In [7]:
#todo: incorporate different collection types rather than a catch all publications, requires other changes to template
publist = {
#     "proceeding": {
#         "file" : "proceedings.bib",
#         "venuekey": "booktitle",
#         "venue-pretext": "In the proceedings of ",
#         "collection" : {"name":"publications",
#                         "permalink":"/publication/"}
        
#     },
    "journal":{
        "file": "mypubs.bib",
        "venuekey" : "journal",
        "venue-pretext" : "",
        "collection" : {"name":"publications",
                        "permalink":"/publication/"}
    } 
}

In [8]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

In [157]:
cts=[]
for pubsource in publist:
    parser = bibtex.Parser()
    bibdata = parser.parse_file(publist[pubsource]["file"])
    entries_sorted=sorted(bibdata.entries,key=lambda x: bibdata.entries[x].fields["year"],reverse=True)
    #loop through the individual references in a given bibtex file
    for ind,bib_id in enumerate(entries_sorted):
        entry=bibdata.entries[bib_id]
        b=entry.fields
        pub_year = f'{b["year"]}'
        #todo: this hack for month and day needs some cleanup
        if "month" in b.keys(): 
            if(len(b["month"])<3):
                pub_month = "0"+b["month"]
                pub_month = pub_month[-2:]
            elif(b["month"] not in range(12)):
                tmnth = strptime(b["month"][:3],'%b').tm_mon   
                pub_month = "{:02d}".format(tmnth) 
            else:
                pub_month = str(b["month"])
        if "day" in b.keys(): 
            pub_day = str(b["day"])
        citation=""
        persons=entry.persons["author"]
        num_of_authors=len(persons)
        author=persons[0]
        citation = author.first_names[0]+" "+author.last_names[0]
        if num_of_authors==2:
            author=persons[1]
            citation = citation+" and "+author.first_names[0]+" "+author.last_names[0]
        for i in range(1,num_of_authors-1):
            author=persons[i]
            citation = citation+", "+author.first_names[0]+" "+author.last_names[0]
        author=persons[-1]
        citation = citation+" and "+author.first_names[0]+" "+author.last_names[0]+"."
        url = b["url"]
        clean_title = b["title"].replace("{", "").replace("}","").replace("\\","")
        citation = citation+"<a href=\""+url+"\">"+clean_title+".</a>"
        journal = entry.fields["journal"]
        citation = citation+"<em> "+journal+"</em> "
        if "arXiv" in journal:
            print(pub_year)
            citation= citation+","+pub_year
        else:
            vol=b["volume"]
            if "number" in b.keys(): 
                num=b["number"]
            else:
                num=""
            pages=b["pages"]

            if "number" in b.keys():
                citation = citation+vol+"("+num+")"+":"+pages+","+pub_year
            else:
                citation= citation+vol+":"+pages+","+pub_year
        cts.append(citation)
        # field may not exist for a reference


2019


In [158]:
all_text="<ol>"
for c in cts:
    all_text+="<li>"+c+"</li>"
all_text+="</ol>"

In [159]:
all_text

'<ol><li>Thomas Schuster, Snir Gazit, Joel Moore and Norman Yao.<a href="https://arxiv.org/abs/1903.02558">Floquet Hopf Insulators.</a><em> arXiv preprint arXiv:1903.02558</em> ,2019</li><li>P Sidorenko, A Dikopoltsev, T Zahavy, O Lahav, S Gazit, Y Shechtman, A Szameit, D Tannor, Y Eldar, M Segev and O Cohen.<a href="http://www.opticsexpress.org/abstract.cfm?URI=oe-27-6-8920">Improving techniques for diagnostics of laser pulses by compact representations.</a><em> Opt. Express</em> 27(6):8920--8934,2019</li><li>Snir Gazit, Fakher Assaad, Subir Sachdev, Ashvin Vishwanath and Chong Wang.<a href="https://www.pnas.org/content/115/30/E6987">Confinement transition of Z$_2$ gauge theories coupled to massless fermions: Emergent quantum chromodynamics and SO(5) symmetry.</a><em> Proceedings of the National Academy of Sciences</em> 115(30):E6987--E6995,2018</li><li>Michael Kolodrubetz, Frederik Nathan, Snir Gazit, Takahiro Morimoto and Joel Moore.<a href="https://link.aps.org/doi/10.1103/PhysRevL

In [10]:
bibdata = parser.parse_file(publist["journal"]["file"])


In [64]:
bibdata.entries['Gazit2009'].fields["journal"]

'Opt. Express'

In [94]:
citation=""
entry=bibdata.entries['Gazit2018']
persons=entry.persons["author"]
num_of_authors=len(persons)
author=persons[0]
citation = author.first_names[0]+" "+author.last_names[0]
if num_of_authors==2:
    citation = citation+" and "+author.first_names[1]+" "+author.last_names[1]
for i in range(1,num_of_authors-1):
    author=persons[i]
    citation = citation+", "+author.first_names[0]+" "+author.last_names[0]
author=persons[-1]
citation = citation+" and "+author.first_names[0]+" "+author.last_names[0]+"."
b=entry.fields
url = b["url"]
clean_title = b["title"].replace("{", "").replace("}","").replace("\\","")
citation = citation+"<a href=\">"+url+"\""+clean_title+".</a>"
journal = entry.fields["journal"]
citation = citation+"<em> "+journal+"</em>"
vol=b["volume"]
num=b["number"]
pages=b["pages"]
pub_year = f'{b["year"]}'
#todo: this hack for month and day needs some cleanup
if "month" in b.keys(): 
    if(len(b["month"])<3):
        pub_month = "0"+b["month"]
        pub_month = pub_month[-2:]
    elif(b["month"] not in range(12)):
        tmnth = strptime(b["month"][:3],'%b').tm_mon   
        pub_month = "{:02d}".format(tmnth) 
    else:
        pub_month = str(b["month"])
if "day" in b.keys(): 
    pub_day = str(b["day"])
citation = citation+vol+"("+num+")"+":"+pages+pub_year

In [95]:
citation

'Snir Gazit, Fakher Assaad, Subir Sachdev, Ashvin Vishwanath and Chong Wang.<a href=">https://www.pnas.org/content/115/30/E6987"Confinement transition of Z$_2$ gauge theories coupled to massless fermions: Emergent quantum chromodynamics and SO(5) symmetry.</a><em> Proceedings of the National Academy of Sciences</em>115(30):E6987--E69952018'

In [84]:
vol

'27'